```{contents}
```
## **Negotiation in LangGraph**

**Negotiation** in LangGraph refers to a structured **multi-agent interaction pattern** where autonomous agents **exchange proposals, evaluate alternatives, resolve conflicts, and converge toward a mutually acceptable decision**, all implemented as a **state-driven cyclic graph**.

This pattern is essential for **distributed reasoning, collaborative problem solving, and consensus-building systems**.

---

### **1. Why Negotiation Is Needed**

Complex tasks involve competing objectives, incomplete information, and uncertainty.

| Scenario             | Why Negotiation                          |
| -------------------- | ---------------------------------------- |
| Multi-agent planning | Different agents propose different plans |
| Resource allocation  | Agents compete for limited resources     |
| Decision validation  | Conflicting evaluations must be resolved |
| System robustness    | Reduces single-agent hallucinations      |

---

### **2. Core Components of Negotiation**

| Component         | Role                               |
| ----------------- | ---------------------------------- |
| Agents            | Independent decision-makers        |
| Proposals         | Candidate solutions                |
| Utility Function  | How each agent evaluates proposals |
| Counter-Proposals | Improved alternatives              |
| Convergence Rule  | When negotiation ends              |
| Arbitration       | Tie-breaking authority             |
| Shared State      | Communication medium               |

---

### **3. Negotiation as a Cyclic Graph**

```
Propose → Evaluate → Counter → Debate → Converge
   ↑                                   ↓
   └────────────── Loop ───────────────┘
```

The loop continues until convergence criteria are satisfied.

---

### **4. State Design for Negotiation**

```python
class NegotiationState(TypedDict):
    proposals: list
    feedback: dict
    round: int
    decision: str
    resolved: bool
```

---

### **5. Minimal Negotiation Workflow**

```python
from langgraph.graph import StateGraph, END

def propose(state):
    state["proposals"].append("Plan A")
    return {"round": state["round"] + 1}

def evaluate(state):
    return {"feedback": {"Plan A": "acceptable"}}

def converge(state):
    if state["round"] >= 2:
        return {"decision": "Plan A", "resolved": True}
    return {"resolved": False}

def route(state):
    return END if state["resolved"] else "propose"

builder = StateGraph(NegotiationState)
builder.add_node("propose", propose)
builder.add_node("evaluate", evaluate)
builder.add_node("converge", converge)

builder.set_entry_point("propose")
builder.add_edge("propose", "evaluate")
builder.add_edge("evaluate", "converge")
builder.add_conditional_edges("converge", route, {"propose": "propose", END: END})

graph = builder.compile()
```

---

### **6. Advanced Negotiation Variants**

| Variant                  | Purpose             |
| ------------------------ | ------------------- |
| Bilateral Negotiation    | Two agents          |
| Multilateral Negotiation | Many agents         |
| Cooperative              | Shared objective    |
| Competitive              | Opposing goals      |
| Consensus                | Majority agreement  |
| Auction-based            | Resource allocation |
| Contract-net             | Task delegation     |

---

### **7. Production-Grade Enhancements**

| Feature           | Function                  |
| ----------------- | ------------------------- |
| Utility scoring   | Quantify proposal quality |
| Timeouts          | Prevent deadlock          |
| Max rounds        | Loop safety               |
| Human arbitration | Final override            |
| Audit logs        | Regulatory compliance     |
| Weighted agents   | Expertise prioritization  |

---

### **8. Use Cases**

| Industry   | Application                   |
| ---------- | ----------------------------- |
| Finance    | Investment strategy selection |
| Healthcare | Treatment plan selection      |
| Legal      | Contract clause agreement     |
| Operations | Resource scheduling           |
| AI Safety  | Cross-agent verification      |

---

### **9. Mental Model**

Negotiation in LangGraph behaves like a **distributed control system**:

> **Proposals → Evaluation → Feedback → Adjustment → Convergence**

This allows systems to achieve **robust, explainable, high-quality decisions**.


### Demonstration


In [1]:
# Complete Negotiation Demo in One Cell (LangGraph)

from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, END

# -------------------------------
# 1. State Definition
# -------------------------------
class NegotiationState(TypedDict):
    proposals: List[str]
    feedback: Dict[str, str]
    round: int
    decision: str
    resolved: bool

# -------------------------------
# 2. Agents (Nodes)
# -------------------------------
def proposer(state):
    proposal = f"Plan-{state['round'] + 1}"
    return {"proposals": state["proposals"] + [proposal], "round": state["round"] + 1}

def evaluator(state):
    current = state["proposals"][-1]
    score = "acceptable" if state["round"] >= 2 else "needs improvement"
    return {"feedback": {current: score}}

def arbiter(state):
    if state["round"] >= 3:
        return {"decision": state["proposals"][-1], "resolved": True}
    return {"resolved": False}

# -------------------------------
# 3. Routing Logic
# -------------------------------
def router(state):
    return END if state["resolved"] else "proposer"

# -------------------------------
# 4. Build Graph
# -------------------------------
builder = StateGraph(NegotiationState)
builder.add_node("proposer", proposer)
builder.add_node("evaluator", evaluator)
builder.add_node("arbiter", arbiter)

builder.set_entry_point("proposer")
builder.add_edge("proposer", "evaluator")
builder.add_edge("evaluator", "arbiter")
builder.add_conditional_edges("arbiter", router, {"proposer": "proposer", END: END})

graph = builder.compile()

# -------------------------------
# 5. Run Negotiation
# -------------------------------
initial_state = {
    "proposals": [],
    "feedback": {},
    "round": 0,
    "decision": "",
    "resolved": False
}

result = graph.invoke(initial_state)
result


{'proposals': ['Plan-1', 'Plan-2', 'Plan-3'],
 'feedback': {'Plan-3': 'acceptable'},
 'round': 3,
 'decision': 'Plan-3',
 'resolved': True}